In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# 1. Load tokenizer dan model IndoBERT
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = BertModel.from_pretrained("indobenchmark/indobert-base-p1")

# 2. Contoh input teks
input_text = "Ini adalah contoh teks."
tokens = tokenizer(input_text, return_tensors="pt")

# 3. Mendapatkan embeddings dari token input
input_embeddings = model.embeddings(tokens['input_ids'])

# 4. Mengakses bobot Q, K, V pada layer pertama dan mencetak bobot awal sebelum update
layer_index = 0
query_weights = model.encoder.layer[layer_index].attention.self.query.weight
key_weights = model.encoder.layer[layer_index].attention.self.key.weight
value_weights = model.encoder.layer[layer_index].attention.self.value.weight

print("Bobot awal Query (sebelum update):", query_weights)
print("Bobot awal Key (sebelum update):", key_weights)
print("Bobot awal Value (sebelum update):", value_weights)

# 5. Menghitung nilai Q, K, V untuk input embeddings
query = torch.matmul(input_embeddings, query_weights.T)
key = torch.matmul(input_embeddings, key_weights.T)
value = torch.matmul(input_embeddings, value_weights.T)

# 6. Perhitungan attention scores
d_k = query.size(-1)  # Dimensi Q atau K
attention_scores = torch.matmul(query, key.transpose(-1, -2)) / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))
attention_probs = F.softmax(attention_scores, dim=-1)

# 7. Menghasilkan attention output
attention_output = torch.matmul(attention_probs, value)

# 8. Feed-forward processing dalam layer encoder (berulang sesuai layer model IndoBERT)
layer_output = model.encoder.layer[layer_index].output.dense(attention_output)

# 9. Mengambil representasi dari token [CLS] untuk klasifikasi
cls_representation = model.pooler.dense(layer_output[:, 0, :])  # Representasi klasifikasi

# 10. Layer klasifikasi sederhana dengan 2 kelas
classification_layer = nn.Linear(model.config.hidden_size, 2)
logits = classification_layer(cls_representation)
predictions = torch.argmax(logits, dim=-1)  # Mengambil prediksi kelas

# 11. Definisikan label target untuk contoh ini
labels = torch.tensor([1])  # Misalnya label kelas positif

# 12. Hitung loss
criterion = nn.CrossEntropyLoss()
loss = criterion(logits, labels)

# 13. Backward pass untuk menghitung gradien
loss.backward()

# 14. Optimizer (misalnya Adam)
optimizer = optim.Adam(model.parameters(), lr=1e-5)

# Melihat bobot sebelum update untuk satu layer
for name, param in model.named_parameters():
    if "encoder.layer.0.attention.self.query.weight" in name:
        print("Bobot Query sebelum update:", param.data)

# 15. Update bobot
optimizer.step()

# 16. Melihat bobot setelah update untuk layer yang sama
for name, param in model.named_parameters():
    if "encoder.layer.0.attention.self.query.weight" in name:
        print("Bobot Query setelah update:", param.data)

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# 1. Load tokenizer dan model IndoBERT
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = BertModel.from_pretrained("indobenchmark/indobert-base-p1")

# 2. Contoh input teks
input_text = "Ini adalah contoh teks."
tokens = tokenizer(input_text, return_tensors="pt")

# 3. Mendapatkan embeddings dari token input
input_embeddings = model.embeddings(tokens['input_ids'])

# 4. Mengakses bobot Q, K, V pada layer pertama dan mencetak bobot awal sebelum update
layer_index = 0
query_weights = model.encoder.layer[layer_index].attention.self.query.weight
key_weights = model.encoder.layer[layer_index].attention.self.key.weight
value_weights = model.encoder.layer[layer_index].attention.self.value.weight

print("Bobot awal Query (sebelum update):", query_weights)
print("Bobot awal Key (sebelum update):", key_weights)
print("Bobot awal Value (sebelum update):", value_weights)

# 5. Menghitung nilai Q, K, V untuk setiap kata dalam input embeddings
query = torch.matmul(input_embeddings, query_weights.T)
key = torch.matmul(input_embeddings, key_weights.T)
value = torch.matmul(input_embeddings, value_weights.T)

# 6. Menghitung skor perhatian antara setiap pasangan kata
d_k = query.size(-1)
attention_scores = torch.matmul(query, key.transpose(-1, -2)) / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))

# 7. Normalisasi softmax pada skor perhatian untuk mendapatkan attention weights
attention_probs = F.softmax(attention_scores, dim=-1)

# 8. Hitung representasi konteksual untuk setiap kata berdasarkan attention weights
contextual_representation = torch.matmul(attention_probs, value)

# 9. Feed-forward processing dalam layer encoder (berulang sesuai layer model IndoBERT)
layer_output = model.encoder.layer[layer_index].output.dense(contextual_representation)

# 10. Mengambil representasi dari token [CLS] untuk klasifikasi
cls_representation = model.pooler.dense(layer_output[:, 0, :])

# 11. Layer klasifikasi sederhana dengan 2 kelas
classification_layer = nn.Linear(model.config.hidden_size, 2)
logits = classification_layer(cls_representation)
predictions = torch.argmax(logits, dim=-1)  # Mengambil prediksi kelas

# 12. Definisikan label target untuk contoh ini
labels = torch.tensor([1])  # Misalnya label kelas positif

# 13. Hitung loss
criterion = nn.CrossEntropyLoss()
loss = criterion(logits, labels)

# 14. Backward pass untuk menghitung gradien
loss.backward()

# 15. Optimizer (misalnya Adam)
optimizer = optim.Adam(model.parameters(), lr=1e-5)

# 16. Logging bobot Q, K, V sebelum update
for name, param in model.named_parameters():
    if "encoder.layer.0.attention.self.query.weight" in name:
        print("Bobot Query sebelum update:", param.data)
    if "encoder.layer.0.attention.self.key.weight" in name:
        print("Bobot Key sebelum update:", param.data)
    if "encoder.layer.0.attention.self.value.weight" in name:
        print("Bobot Value sebelum update:", param.data)

# 17. Update bobot
optimizer.step()

# 18. Logging bobot Q, K, V setelah update
for name, param in model.named_parameters():
    if "encoder.layer.0.attention.self.query.weight" in name:
        print("Bobot Query setelah update:", param.data)
    if "encoder.layer.0.attention.self.key.weight" in name:
        print("Bobot Key setelah update:", param.data)
    if "encoder.layer.0.attention.self.value.weight" in name:
        print("Bobot Value setelah update:", param.data)